Baseado em https://github.com/aksh-ai/neuralBlack/blob/master/torch_brain_tumor_classifier.ipynb

E https://iopscience.iop.org/article/10.1088/1361-6560/aac71a/pdf

## Import Necessary Libraries

We'll be using the following libraries to complete our classification problem:

* Numpy - For linear algebra operations 
* Torch - Pytorch Deep Learning Framework
* Torch NN - Neural network class from Pytorch library
* Torch NN Functional - Functional Neural Network class from Pytorch library
* Torch Utils Data: DataLoader, Dataset - Dataset class used to create custom dataset class by subclassing it and DataLoader is used to laod data in batches using dataset class in real-time.
* Torchvision: Transforms, Models - Trochvision provides augmentation techniques using transforms class and transfer learning models are available in models class
* OS - To use Operating System methods
* Random - To set random seed at specific places where random operations take place just so it happens the same way everytime it is executed
* Pandas - To create DataFrame, CSV files, etc
* Time - To perform date time operations
* Seaborn - For sophisticated visualization
* Pickle - To save and load binary files of our training data
* Scikit-Learn - For evaluating our Classifier and for cross-validation split
* Matplotlib - To visualize images, losses and accuracy
* Google Colab Drive - To mount Google Drive so we can perform storage and loading operations using it
                      
Let's go ahead and import the required libraries

In [1]:
!pip install torchio
!pip install torch#==1.9.0
!pip install torchvision
!pip install seaborn
!pip install scikit-learn
!pip install opencv-python
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from torchvision.utils import make_grid
import os
import random
import numpy as np
import pandas as pd
import pickle
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image
import sys
import cv2
import torchio as tio

import k_space, subsampling, radial
!pwd

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Invalid requirement: 'torch#==1.9.0'
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
/home/jonathan/Dropbox/UnB/unet_mri_reconstruction


Import Google Drive for persistent storage of our training data, neural network model weights and other required files.

In [2]:
from skimage.transform import resize
from skimage.data import shepp_logan_phantom
import matplotlib.pyplot as plt 

def sampling_pattern(sampling,imsize):
    if sampling==0:
        idx=[i for i in range(0,imsize[0]*imsize[1])]
    elif sampling==1:
        idx=subsampling.vertical_subsampling(imsize, rate=2, low_freq=0)
    elif sampling==2:
        idx=subsampling.vertical_subsampling(imsize, rate=2, low_freq=12/100)
    elif sampling==3:
        idx=subsampling.vertical_subsampling(imsize, rate=4, low_freq=0)
    elif sampling==4:
        idx=subsampling.vertical_subsampling(imsize, rate=4, low_freq=12/100)
    elif sampling==5:
        idx=radial.radial2D(20,imsize)
    elif sampling==6:
        idx=radial.radial2D(40,imsize)
    elif sampling==7:
        idx=radial.radial2D(60,imsize)
    elif sampling==8:
        idx=radial.radial2D(80,imsize)
    elif sampling==9:
        idx=radial.radial2D(100,imsize)
    else:
        return False
    return idx
    
def l2_rec_preset(image,num=2):
    imsize=image.shape
    idx=sampling_pattern(num,imsize)
    rec=k_space.l2_rec(image,idx)
    return rec


image = shepp_logan_phantom()  
imsize=(256,256)
image=resize(image,imsize)
def all_funs():
    funs=[]
    plt.figure(figsize = (20,10))

    for i in range(10):
        f=lambda im,i=i: l2_rec_preset(im,num=i)
        funs.append(f)
        plt.subplot(2, 5, i+1)
        plt.imshow(f(image), cmap = 'gray')        
    plt.show()
    return funs
funs=all_funs()
    
'''
MAX=5
plus_n=[lambda x: x+i for i in range(MAX)]

plus_n_=[]
for i in range(MAX):
    plus_n_.append(lambda x: x+i)

def all_plus_n():
    plus_ns=[]
    for i in range(MAX):
        plus_ns.append(lambda x: x+i)
    return plus_ns
plus_n__=all_plus_n()    

for i in range(len(plus_n)):
    print('plus_n[{}]({})={}'.format(i,3,plus_n[i](3)))
    print('plus_n_[{}]({})={}'.format(i,3,plus_n_[i](3)))
    print('plus_n__[{}]({})={}'.format(i,3,plus_n__[i](3)))
    print()
'''


ModuleNotFoundError: No module named 'skimage'

In [ ]:

image = shepp_logan_phantom()  
imsize=(256,256)
image=resize(image,imsize)

recs=[]
plt.figure(figsize = (20,10))


for i,f in enumerate(funs):
    rec=f(image)
    recs.append(rec)
    plt.subplot(2, 5, i+1)
    plt.imshow(f(image), cmap = 'gray')
plt.show()
              

In [ ]:
# This values defines the dimensions of the resized image used in this notebook.
# It's recommended to use (1,256,256) images, as colab doesn't have enough memory
# to process bigger images. Resizing the images to this resolution didn't have 
# much of an impact in the traning results
img_channels = 1
img_size = 256

## Custom Dataset Class

Create a custom dataset class that augments each image into 8 different angles: 0, 45, 90, 120, 180, 270, 300, 330 degrees. Fuse it with Pytorch's DataLoader class so data can be loaded, augmented and trained in realtime instead of caching all training samples in memory for augmenting.

In [ ]:
import k_space
class L2rec:
    def __init__(self, sampling):
        self.sampling=sampling

    def __call__(self, image):
        im=np.array(image)
        imsize=im.shape
        idx=sampling_pattern(self.sampling,imsize)
        rec=k_space.l2_rec(im,idx)
        
        return Image.fromarray(rec)

    def __repr__(self):
        repr = f"{self.__class__.__name__  }(sampling={self.sampling})"
        return repr
l=L2rec(1) 
l.__repr__()

In [ ]:
def defineRecTransform(pattern=0,img_size=(256,256)):
  funs=all_funs()
  transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),         
        transforms.Resize(img_size),
        L2rec(sampling=pattern),
        transforms.ToTensor()                                 
    ])
  return transform

def defineTransform(img_size=(256,256)):
  transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),         
        transforms.Resize(img_size),
        transforms.ToTensor()                                 
    ])
  return transform

class BrainTumorDataset(Dataset):
  def __init__(self, dataset, img_size=(256,256), sampling_pattern=2):
    self.img_size = img_size
    self.dataset = dataset
    self.transform = defineTransform()
    self.rectransform = defineRecTransform(pattern=sampling_pattern)
  def __len__(self):
    # return length of image samples    
    return len(self.dataset)

  def __getitem__(self, idx):
    X, y = self.dataset[idx]
    # perform transformations on one instance of X
    # Original image as a tensor
    data = self.transform(X)
    recdata = self.rectransform(X)
    # store the images in a list
    new_batch = [data]
    new_batch = new_batch[:]
    new_batch_rec = [recdata]
    return (torch.stack(new_batch_rec),torch.stack(new_batch))

Load manually separated dataset with ImageFolder class

In [ ]:
from torchvision.datasets import ImageFolder

dataTreino = ImageFolder('./patientImages/splits/train')
dataValid = ImageFolder('./patientImages/splits/valid')
dataTeste = ImageFolder('./patientImages/splits/test')

Using ImageFolder data to feed our custom dataset

In [ ]:
train_set = BrainTumorDataset(dataTreino,sampling_pattern=2)
valid_set = BrainTumorDataset(dataValid,sampling_pattern=2)
test_set = BrainTumorDataset(dataTeste,sampling_pattern=2)


Print original number of samples in each set

In [ ]:
print(f"Number of training samples: {len(train_set)}")
print(f"Number of validation samples: {len(valid_set)}")
print(f"Number of testing samples: {len(test_set)}")

Create a DataLoader for each set with batch size of 4 and shuffling enabled

In [ ]:
# Defines batch size. It's important to keep a low number in train, so the 
# program doesn't crash for lack of memory
train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8

In [ ]:
train_gen = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True, num_workers=2, drop_last=True)
valid_gen = DataLoader(valid_set, batch_size=valid_batch_size, shuffle=True, pin_memory=True, num_workers=2, drop_last=True)
test_gen = DataLoader(test_set, batch_size=test_batch_size, shuffle=True, pin_memory=True, num_workers=2, drop_last=True)


Get device to set the training to run on GPU or CPU later based on its availibility

In [ ]:
device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)
print(device_name)

## Build the Model

* Unet

In [ ]:
class Unet(nn.Module):    
    def __init__(self): # imsize=[1,256,256]
        super().__init__()
        #o = [i + 2*p - k - (k-1)*(d-1)]/s + 1
        #o=[256+(2*1)-3-(2*0)]/2 +1=
        self.encoder_1 = nn.Sequential(
            nn.Conv2d(1, 64, 3, stride=1, padding=1,dilation=1), #in: 1,256,256, out: 64,256,256
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),            
            #maxpooling
            nn.Conv2d(64, 64, 3, stride=1, padding=1), #in: 64,256,256, out: 64,256,256
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            #maxpooling
        )
        self.encoder_2 = nn.Sequential(
            nn.Conv2d(64, 64, 3, stride=2, padding=1), #in: 64,256,256, out: 64,128,128
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),            
            #maxpooling
            nn.Conv2d(64, 128, 3, stride=1, padding=1), #in: 64,128,128, out: 128,128,128
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            #maxpooling
            nn.Conv2d(128, 128, 3, stride=1, padding=1), #in:128,128,128, out: 128,128,128
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            #max
        )
        #input: 128,64,64
        self.encoder_3 = nn.Sequential(
            nn.Conv2d(128, 128, 3, stride=2, padding=1), #in: 128,128,128, out: 128,64,64
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),            
            #maxpooling
            nn.Conv2d(128, 256, 3, stride=1, padding=1), #in: 128,64,64, out: 256,64,64
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
        )
        self.decoder_1 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, stride=1, padding=1, output_padding=0), #in: 256,64,64, out: 128,64,64
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 128, 3, stride=2, padding=1, output_padding=1),#in: 128,64,64, out: 128,128,128
            nn.BatchNorm2d(128),
            nn.ReLU(True)
        )
        self.decoder_2 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, stride=1, padding=1, output_padding=0), #in: 256,128,128, out: 128,128,128
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 3, stride=1, padding=1, output_padding=0),#in: 128,128,128, out: 64,128,128
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64,64, 3, stride=2, padding=1, output_padding=1),#in: 64,128,128, out: 64,256,256
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )
        self.decoder_3 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 3, stride=1, padding=1, output_padding=0), #in: 128,256,256, out: 64,256,256
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 64, 3, stride=1, padding=1, output_padding=0),#in: 64,256,256 out: 64,256,256
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 1, 3, stride=1, padding=1, output_padding=0),#in: 64,256,256 out: 1,256,256
            nn.BatchNorm2d(1),
            nn.ReLU(True)
        )

    def forward(self, x):
        x1 = self.encoder_1(x)
        x2 = self.encoder_2(x1)
        x3 = self.encoder_3(x2)
        x4 = self.decoder_1(x3)
        x5 = self.decoder_2(torch.cat((x4,x2), 1))
        x6 = self.decoder_3(torch.cat((x5,x1), 1))

        return x6
        
def testUnet(gen):
  unet = Unet()
  print(type(gen))
  for data in iter(gen):
      x=data[0]
      print(type(data))
      x=x.view(-1, 1, 256,256)
      print('x shape:', x.shape)
      x1 = unet.encoder_1(x)
      print('x1 shape:', x1.shape)
      x2 = unet.encoder_2(x1)
      print('x2 shape:', x2.shape)
      x3 = unet.encoder_3(x2)
      print('x3 shape:', x3.shape)
      x4 = unet.decoder_1(x3)
      print('x4 shape:', x4.shape)
      print('x4 cat x2 shape:', torch.cat((x4,x2), 1).shape)      
      x5 = unet.decoder_2(torch.cat((x4,x2), 1))
      print('x5 shape:', x5.shape)
      print('x5 cat x1 shape:', torch.cat((x5,x1), 1).shape)
      x6 = unet.decoder_3(torch.cat((x5,x1), 1))
      print('x6 shape:', x6.shape)

      break
  
testUnet(test_gen)

unet=Unet()
# set all paramters as trainable
for param in unet.parameters():
    param.requires_grad = True

# set model to run on GPU or CPU absed on availibility
unet.to(device)


## Set Training Configuration

* Set model's loss function as MSE

* Set RMSprop optimizer with 0.9 weigth decay and learning rate 0.001 as the model's optimizer.

* Create empty lists to store training losses, validation losses

In [ ]:
# loss function
# if GPU is available set loss function to use GPU
#optimizer = torch.optim.Adam(unet.parameters(), lr=1e-3)
optimizer = torch.optim.RMSprop(unet.parameters(), lr=0.001, weight_decay=0.9)
# mean-squared error loss
criterion = nn.MSELoss()


# number of training iterations
epochs = 2000

# empty lists to store losses
train_losses = []
val_losses = []


## Util function

### Checkpoint Saver
A function to save the model using checkpoints based on best loss achieved during every iteration compared with previous iteration's loss. We'll load the checkpoint and resume training in case Colab's runtime get's disconnected due to inactivity or any other issues.

In [ ]:
PATH='./unet_ckpt1.pth.tar'
def save_checkpoint(epoch, model, optimizer, loss, best_prec1, train_losses, val_losses, is_best, filename=PATH):
    if is_best:
        d={ 'epoch': i + 1,
            'state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            'train_losses': train_losses,
            'val_losses': val_losses,
            'best_prec1': best_prec1
          }
        torch.save(d, filename)
        


## Train the model

In [ ]:
# set training start time
start_time = time.time()
from tqdm.notebook import tqdm_notebook
from tqdm import tqdm
# set best_prec loss value as 2 for checkpoint threshold
best_prec1 = 999999
epoch=0

if os.path.isfile(PATH):
    checkpoint = torch.load(PATH)
    unet.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    train_losses= checkpoint['train_losses']
    val_losses= checkpoint['val_losses']
    best_prec1 = checkpoint['best_prec1']
    print('loading checkpoint', PATH)
    

# empty batch variables
b = None
train_b = None
test_b = None
img_channels, img_size=1,256

# start training
while epoch<epochs:
    epoch+=1
    print('epoch:', epoch)
    unet.train()
    # empty training correct and test correct counter as 0 during every iteration
    trn_corr = 0
    tst_corr = 0
    
    # contador de imageis totais
    total_images = 0
    
    # set epoch's starting time
    e_start = time.time()
    
    # train in batches
    for b, (x, y) in enumerate(train_gen):
        if b%10==0:
            print('{}/{}, time: {:.2f}min'.format(b,len(train_gen),(time.time()-start_time)/60),end='| ')
        # set label as cuda if device is cuda
        x, y = x.to(device), y.to(device)
        x=x.view(-1, img_channels, img_size, img_size)
        # forward pass image sample
        y_pred = unet(x)
        
        #y_pred=decoder latente)
        # calculate loss
        loss = criterion(x.float(), y_pred.float())

        # set optimizer gradients to zero
        optimizer.zero_grad()
        # back propagate with loss
        loss.backward()
        # perform optimizer step
        optimizer.step()

    # set epoch's end time
    e_end = time.time()
    # print training metrics
    print(f'Epoch {(i+1)}\n  Loss: {loss.item():2.4f}  Duration: {((e_end-e_start)/60):.2f} minutes') # total_images = 4 images per batch * 8 augmentations per image * batch length

    # some metrics storage for visualization
    train_losses.append(loss)
    
    x, y = None, None

    # validate using validation generator
    # do not perform any gradient updates while validation
    unet.eval()
    with torch.no_grad():
        for b, (x, y) in enumerate(valid_gen):
            # set label as cuda if device is cuda
            x, y = x.to(device), y.to(device)
            x=x.view(-1, img_channels, img_size, img_size)
            # forward pass image
            y_val = unet(x)
            #y_val = decoder(latente)
            if b==0:
              plt.figure(figsize = (10,20))
              plt.subplot(1, 2, 1),plt.imshow(x.detach().cpu().numpy()[0,0,:,:], cmap = 'gray')
              plt.subplot(1, 2, 2),plt.imshow(y_val.detach().cpu().numpy()[0,0,:,:], cmap = 'gray')
              #plt.subplot(1, 3, 3),plt.imshow(y.detach().cpu().numpy()[0,0,:,:], cmap = 'gray')
              #plt.title(f'Imagem original (resized {img_size}x{img_size})'), plt.xticks([]), plt.yticks([])
              plt.show()
              



    # get loss of validation set
    loss = criterion(x.float(), y_val.float())
    # print validation metrics
    print(f'Validation Loss: {loss.item():2.4f}\n')
    
    # some metrics storage for visualization
    test_b  = b
    val_losses.append(loss)
    
    # if current validation loss is less than previous iteration's validatin loss create and save a checkpoint
    is_best = loss < best_prec1
    best_prec1 = min(loss, best_prec1)
    save_checkpoint(i + 1,unet, optimizer, loss, best_prec1, train_losses, val_losses, is_best)



# set total training's end time
end_time = time.time() - start_time    

# print training summary
print("\nTraining Duration {:.2f} minutes".format(end_time/60))
print("GPU memory used : {} kb".format(torch.cuda.memory_allocated()))
print("GPU memory cached : {} kb".format(torch.cuda.memory_cached()))

In [ ]:
torch.cuda.empty_cache()

## Save the model

Save the model after the training is completed

**Lembre-se de mudar o nome do arquivo**

In [ ]:
torch.save(unet.state_dict(), './unet_model.pt')

## Evaluation (com imagens sem degradações)

Print the validation accuracy of the model calculated using validation set during training 

Plot the loss graph

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.title('Loss Metrics')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()
plt.show()

Empty out training set and validation set to free up RAM / Cache

In [ ]:
# unet.load_state_dict(torch.load('/content/drive/MyDrive/2021-BRICS/unet_model.pt'))

train_gen = None
valid_gen = None
train_set = None
valid_set = None

Set model to evaluation mode

Calculate loss, correctly classified samples, predicted values, labels and store them in a list using test dataloader

In [ ]:
# set model to evaluation mode
unet.eval()

# perform no gradient updates
with torch.no_grad():
    # soem metrics storage for visualization and analysis
    correct = 0
    test_loss = []

    # contador de imageis totais
    total_images = 0
    # perform test set evaluation batch wise
    for (x, y) in test_gen:
        # set label to use CUDA if available
        x, y = x.to(device), y.to(device)
        x=x.view(-1, img_channels, img_size, img_size)
        
        # forward pass image sample
        y_pred = unet(x)
        
        # calculate loss
        loss = criterion(x.float(), y_pred.float())

        test_loss.append(loss)
        
print(f"Test Loss: {test_loss[-1].item():.4f}")

#Fim